In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_MN = pd.read_html('https://en.wikipedia.org/wiki/Minnesota')

In [ ]:
#The unique point here is that table_MN is a list of all the tables on the page:

In [2]:
print(f'Total tables: {len(table_MN)}')

Total tables: 28


In [ ]:
''''''With 38 tables, it can be challenging to find the one you need. 
To make the table selection easier, use the match parameter to select a 
subset of tables. We can use the caption “Election results from statewide 
races” to select the table:''''''

In [3]:
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Minnesota', match='Election results from statewide races')
len(table_MN)

1

In [4]:
df = table_MN[0]
df.head()

,Year,Office,GOP,DFL,Others
0,2020,President,45.3%,52.4%,2.3%
1,2020,Senator,43.5%,48.8%,7.7%
2,2018,Governor,42.4%,53.9%,3.7%
3,2018,Senator,36.2%,60.3%,3.4%
4,2018,Senator,42.4%,53.0%,4.6%


In [ ]:
'''Pandas makes it easy to read in the table and also handles the year 
column that spans multiple rows. This is an example where it is easier
to use pandas than to try to scrape it all yourself.

Overall, this looks ok until we look at the data types with df.info() :'''

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
Year      26 non-null int64
Office    26 non-null object
GOP       26 non-null object
DFL       26 non-null object
Others    26 non-null object
dtypes: int64(1), object(4)
memory usage: 1.1+ KB


In [7]:
'''We need to convert the GOP, DFL and Other columns to numeric values 
if we want to do any analysis.'''
df['GOP'].astype('float')

#If we try:

ValueError: could not convert string to float: '45.3%'

In [ ]:
#We get an error:
#ValueError: could not convert string to float: '45.3%'

In [8]:
df['GOP'].replace({'%':''}, regex=True).astype('float')

0     45.3
1     43.5
2     42.4
3     36.2
4     42.4
5     44.9
6     44.5
7     42.9
8     45.1
9     30.6
10    43.2
11    43.8
12    42.0
13    46.7
14    37.9
15    47.6
16    44.4
17    49.5
18    45.5
19    43.3
20    34.3
21    35.0
22    41.3
23    63.3
24    49.1
25    31.9
Name: GOP, dtype: float64

In [ ]:
'''Note, that I had to use the regex=True parameter for this to work since the % is a part of 
the string and not the full string value.

Now, we can call replace all the % values and convert to numbers using 
pd.to_numeric() and apply()'''

In [9]:
df = df.replace({'%': ''}, regex=True)
df[['GOP', 'DFL', 'Others']] = df[['GOP', 'DFL', 'Others']].apply(pd.to_numeric)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
Year      26 non-null int64
Office    26 non-null object
GOP       26 non-null float64
DFL       26 non-null float64
Others    26 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.1+ KB


In [10]:
df.head()

,Year,Office,GOP,DFL,Others
0,2020,President,45.3,52.4,2.3
1,2020,Senator,43.5,48.8,7.7
2,2018,Governor,42.4,53.9,3.7
3,2018,Senator,36.2,60.3,3.4
4,2018,Senator,42.4,53.0,4.6


In [ ]:
#How to: Parse an HTML table and write to a CSV


In [12]:
from bs4 import BeautifulSoup
import csv
html = open("table.html").read()
soup = BeautifulSoup(html)
table = soup.find("table")

output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
    
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)